# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities = pd.read_csv("../WeatherPy/output_data/cities.csv")
cities.head()

,Unnamed: 0,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mataura,-46.1927,168.8643,49.48,76,35,2.93,NZ,1622509855
1,1,jamestown,42.0970,-79.2353,58.32,54,1,5.75,US,1622509828
2,2,mehamn,71.0357,27.8492,41.18,96,100,4.03,NO,1622509855
3,3,busselton,-33.6500,115.3333,52.09,84,50,6.08,AU,1622509856
4,4,pangnirtung,66.1451,-65.7125,28.49,92,73,4.05,CA,1622509856


In [3]:
cities.describe()

,Unnamed: 0,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,571.000000,571.000000,571.000000,571.000000,571.000000,571.000000,571.000000,5.710000e+02
mean,285.000000,20.793214,15.587273,65.235972,70.259194,57.451839,7.410508,1.622510e+09
std,164.977776,33.290560,91.296231,15.509278,21.415261,40.441281,5.027271,7.145136e+01
min,0.000000,-54.800000,-179.166700,24.330000,6.000000,0.000000,0.000000,1.622510e+09
25%,142.500000,-7.094300,-65.748850,54.130000,59.500000,13.000000,3.440000,1.622510e+09
50%,285.000000,25.155800,18.944500,66.000000,75.000000,75.000000,5.930000,1.622510e+09
75%,427.500000,49.991750,96.749950,77.630000,85.000000,100.000000,10.660000,1.622510e+09
max,570.000000,78.218600,179.316700,104.320000,100.000000,100.000000,24.180000,1.622510e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key= g_key)

In [5]:
locations = cities[["Lat", "Lon"]]
humidity= cities["Humidity"]
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, point_radius=5))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.


A max temperature lower than 80 degrees but higher than 70.

Wind speed less than 10 mph.

Zero cloudiness.

Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.

Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.

In [6]:
narrowed_cities_df = cities.loc[
    (cities["Max Temp"] < 80) &
    (cities["Max Temp"] > 70) &
    (cities["Wind Speed"]<10)&
    (cities["Cloudiness"] == 0)
]
narrowed_cities_df

,Unnamed: 0,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
33,33,gat,31.6100,34.7642,72.39,100,0,0.63,IL,1622509866
135,135,hasaki,35.7333,140.8333,72.28,71,0,7.54,JP,1622509900
309,309,kogon,39.7228,64.5517,77.04,36,0,4.61,UZ,1622509958
331,331,sirjan,29.4520,55.6814,74.23,14,0,5.12,IR,1622509965
477,477,takefu,35.9039,136.1669,71.29,50,0,2.55,JP,1622510013
479,479,urumqi,43.8010,87.6005,77.95,21,0,1.99,CN,1622510013


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = narrowed_cities_df[["City", "Country", "Lat", "Lon"]].copy()
hotel_df["Hotel Name"] = None
hotel_df.head()

,City,Country,Lat,Lon,Hotel Name
33,gat,IL,31.6100,34.7642,None
135,hasaki,JP,35.7333,140.8333,None
309,kogon,UZ,39.7228,64.5517,None
331,sirjan,IR,29.4520,55.6814,None
477,takefu,JP,35.9039,136.1669,None


In [8]:
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

for i, row in hotel_df.iterrows():
    params["location"] = "{},{}".format(row["Lat"], row["Lon"])
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    try:
        response = requests.get(base_url, params=params)
        data = response.json()
        hotel_df.loc[i, "Hotel Name"] = data["results"][0]["name"]
    except:
        print("Skipping for city {}".format(hotel_df.loc[i, "City"]))
hotel_df.dropna(inplace=True)
hotel_df

,City,Country,Lat,Lon,Hotel Name
33,gat,IL,31.6100,34.7642,OlusHome
135,hasaki,JP,35.7333,140.8333,Hotel Sunrise Choshi
309,kogon,UZ,39.7228,64.5517,"""AFROSIYOB"" HOSTEL"
331,sirjan,IR,29.4520,55.6814,MAPNA Building
477,takefu,JP,35.9039,136.1669,Hotel CrownHills Takefu
479,urumqi,CN,43.8010,87.6005,Hua Ling Grand Hotel Urumqi


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [10]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))